In [1]:
import urllib.request as request
import os
import re
import string
import tiktoken

import torch

In [3]:
torch.cuda.is_available()

False

In [38]:
# if not os.path.exists('data/verdict.txt'):
#     file_path = 'theverdict.txt'
#     url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/refs/heads/main/ch02/01_main-chapter-code/the-verdict.txt")
#     request.urlretrieve(url, file_path)

In [39]:
with open('theverdict.txt','r', encoding='utf-8') as file:
    text = file.read()

## Tokenizer 

In [42]:
tokenizer = tiktoken.get_encoding('gpt2')

In [43]:
tokenizer.encode('hai , wwhar <|end|>', allowed_special={'<|endoftext|>'})

[44488, 837, 266, 1929, 283, 1279, 91, 437, 91, 29]

## Data Sampling with sliding window

In [46]:
enc_token = tokenizer.encode(text)

In [55]:
context_len = 4
for i in range(5):
    
    input_token = enc_token[i :i + context_len]
    labels = enc_token[i+1 : i+context_len+1]
    
    print(f"input = {input_token}")
    print(f"labels =    {labels}")


input = [40, 367, 2885, 1464]
labels =    [367, 2885, 1464, 1807]
input = [367, 2885, 1464, 1807]
labels =    [2885, 1464, 1807, 3619]
input = [2885, 1464, 1807, 3619]
labels =    [1464, 1807, 3619, 402]
input = [1464, 1807, 3619, 402]
labels =    [1807, 3619, 402, 271]
input = [1807, 3619, 402, 271]
labels =    [3619, 402, 271, 10899]


## Coding Attention mechanism|

In [143]:
class SelfAttention(torch.nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        self.q_W = torch.nn.Linear(d_model, d_model)
        self.k_W = torch.nn.Linear(d_model, d_model)
        self.v_W = torch.nn.Linear(d_model, d_model)
        self.o_W = torch.nn.Linear(d_model, d_model)
        
    def forward(self, x):
        query = self.q_W(x)
        key = self.k_W(x)
        value = self.v_W(x)
        attention_scores = query@key.T
        attention_scores = torch.where(torch.tril(input=attention_scores)==0,torch.tensor(float('-inf')),torch.tril(input=attention_scores))
        print(f"before applying softmax:")
        print(attention_scores)
        attention_scores = torch.softmax(attention_scores, dim=-1)
        print(f"after softmax")
        print(attention_scores)
        contextualized_embedding = attention_scores@value
        return contextualized_embedding
        
        

In [144]:
torch.tril(input=sample)

tensor([[0.4963, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6341, 0.4901, 0.0000, 0.0000, 0.0000],
        [0.3489, 0.4017, 0.0223, 0.0000, 0.0000],
        [0.5185, 0.6977, 0.8000, 0.1610, 0.0000]])

In [145]:
torch.manual_seed(0)
sample = torch.rand((4,5))
print(sample)

tensor([[0.4963, 0.7682, 0.0885, 0.1320, 0.3074],
        [0.6341, 0.4901, 0.8964, 0.4556, 0.6323],
        [0.3489, 0.4017, 0.0223, 0.1689, 0.2939],
        [0.5185, 0.6977, 0.8000, 0.1610, 0.2823]])


In [146]:
attention = SelfAttention(sample.shape[1])

In [147]:
sample.transpose(-1, -2)

tensor([[0.4963, 0.6341, 0.3489, 0.5185],
        [0.7682, 0.4901, 0.4017, 0.6977],
        [0.0885, 0.8964, 0.0223, 0.8000],
        [0.1320, 0.4556, 0.1689, 0.1610],
        [0.3074, 0.6323, 0.2939, 0.2823]])

In [148]:
attention(sample)

before applying softmax:
tensor([[ 0.0922,    -inf,    -inf,    -inf],
        [ 0.2716,  0.1647,    -inf,    -inf],
        [-0.0206, -0.4406, -0.0593,    -inf],
        [ 0.1061, -0.1440,  0.0595,  0.0009]], grad_fn=<WhereBackward0>)
after softmax
tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5267, 0.4733, 0.0000, 0.0000],
        [0.3818, 0.2509, 0.3673, 0.0000],
        [0.2752, 0.2143, 0.2627, 0.2478]], grad_fn=<SoftmaxBackward0>)


tensor([[ 0.0873, -0.5075, -0.1755, -0.1140,  0.3443],
        [ 0.0824, -0.5954, -0.1146, -0.2196,  0.1748],
        [ 0.0391, -0.5315, -0.1859, -0.1900,  0.1965],
        [ 0.0994, -0.5635, -0.1211, -0.1661,  0.1906]], grad_fn=<MmBackward0>)

In [149]:
torch_x = torch.tensor([[ 9.2212e-02,  -1.0000e-18,  -1.0000e-18,  -1.0000e-18],
                        [ 2.7157e-01,  1.6467e-01,  1.0000e-18,  1.0000e-18],
                        [-2.0602e-02, -4.4060e-01, -5.9342e-02,  1.0000e-18],
                        [ 1.0613e-01, -1.4402e-01,  5.9514e-02,  9.3780e-04]])

In [150]:
torch.softmax(torch_x, dim=1)

tensor([[0.2677, 0.2441, 0.2441, 0.2441],
        [0.2921, 0.2625, 0.2227, 0.2227],
        [0.2747, 0.1805, 0.2643, 0.2805],
        [0.2752, 0.2143, 0.2627, 0.2478]])

In [122]:
import torch.nn.functional as f

In [130]:
f.softmax(torch_x, dim=1)

tensor([[0.2677, 0.2441, 0.2441, 0.2441],
        [0.2921, 0.2625, 0.2227, 0.2227],
        [0.2747, 0.1805, 0.2643, 0.2805],
        [0.2752, 0.2143, 0.2627, 0.2478]])

In [125]:
torch_x

tensor([[ 9.2212e-02, -1.0000e-18, -1.0000e-18, -1.0000e-18],
        [ 2.7157e-01,  1.6467e-01,  1.0000e-18,  1.0000e-18],
        [-2.0602e-02, -4.4060e-01, -5.9342e-02,  1.0000e-18],
        [ 1.0613e-01, -1.4402e-01,  5.9514e-02,  9.3780e-04]])